In [1]:
import pickle
import keras
import tensorflow as tf
#from keras import backend as K
import numpy as np
import sys
import os
sys.path.append(os.path.abspath('../'))
import helpers
import helpers.data_generator
from helpers.data_generator import process_data, DataGenerator
from helpers.custom_losses import denorm_loss, hinge_mse_loss
from helpers.custom_losses import percent_correct_sign, baseline_MAE
# from models.LSTMConv2D import get_model_lstm_conv2d, get_model_simple_lstm
# from models.LSTMConv2D import get_model_linear_systems, get_model_conv2d
#from utils.callbacks import CyclicLR, TensorBoardWrapper
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from time import strftime, localtime
import matplotlib
from matplotlib import pyplot as plt
import copy
%matplotlib inline
from helpers.normalization import normalize, denormalize, renormalize
from tqdm import tqdm
from sklearn import preprocessing
from sklearn import decomposition

Using TensorFlow backend.
/home/lzfang/.conda/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lzfang/.conda/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lzfang/.conda/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lzfang/.conda/envs/tfgpu/l

In [2]:
# full_data_path = '/scratch/gpfs/jabbate/old_stuff/new_data/final_data.pkl'
full_data_path = '/scratch/gpfs/jabbate/full_data/train_data.pkl'

test_data_path = '/scratch/gpfs/jabbate/full_data/test_data.pkl' 
# profiles = ['thomson_temp_EFITRT1','thomson_dens_EFITRT1','temp','dens']
profiles = ['temp','dens','rotation','itemp','idens','press_EFIT01','q_EFIT01','press_EFIT02','q_EFIT02',]
# for profile in tqdm(profiles):
traindata, valdata, normalization_dict = helpers.data_generator.process_data(full_data_path,
                                                      profiles,
                                                      normalization_method='RobustScaler',
                                                      window_length=1,
                                                      window_overlap=0,
                                                      lookbacks=0,
                                                      lookahead=4,
                                                      sample_step=1,
                                                      uniform_normalization=True,
                                                      train_frac=1,
                                                      val_frac=0,
                                                      nshots=10000,
                                                      verbose=2,
                                                      flattop_only=True,
                                                      randomize=False,
                                                      pruning_functions=['remove_nan',
                                                                         'remove_dudtrip',
                                                                         'remove_I_coil',
                                                                         'remove_non_gas_feedback',
                                                                         'remove_ECH','remove_outliers'],
                                                      excluded_shots=['topology_TOP', 
                                                                      'topology_OUT',
                                                                      'topology_MAR',
                                                                      'topology_IN',
                                                                      'topology_DN',
                                                                      'topology_BOT'],
                                                      delta_sigs=[],
                                                      invert_q=True,
                                                      val_idx = 0)

Loading
Signals: dens, idens, itemp, press_EFIT01, press_EFIT02, q_EFIT01, q_EFIT02, rotation, temp
Number of useable shots:  4406
Number of shots used:  4406
Total number of timesteps:  470664
Shots with Complete NaN: 
320950 samples total
Removing ECH
Removed 131668 samples
189282 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 35361 samples
153921 samples remaining
Removing weird I-coils
Removed 33051 samples
120870 samples remaining
Removing NaN
Removed 0 samples
120870 samples remaining
Removing dudtrip
Removed 5539 samples
115331 samples remaining
Removing jagged q profiles
Removed 82 samples
115249 samples remaining
Removing zero q profiles
Removed 8126 samples
107123 samples remaining
Removing outlier pressure profiles
Removed 46 samples
107077 samples remaining
Removing negative pressure profiles
Removed 10909 samples
96168 samples remaining
Removing negative pressure profiles
Removed 2551 samples
93617 samples remaining
93617 samples remaining after pruning


In [6]:
traindata = denormalize(traindata,normalization_dict)

Denormalizing: 100%|##########| 18/18 [00:00<00:00, 32.05it/s]


In [7]:
train_generator = DataGenerator(traindata,
                                1,
                                profiles,
                                [],
                                profiles,
                                [],
                                {profile:0 for profile in profiles},
                                4,
                                True,
                                2,
                                False,
                                sample_weights=None)

Loading
Signals: curr_target, dens, density_estimate, kappa_EFIT01, li_EFIT01, pinj, press_EFIT01, q_EFIT01, rotation, target_density, temp, tinj, triangularity_bot_EFIT01, triangularity_top_EFIT01, volume_EFIT01
Number of useable shots:  1116
Number of shots used:  1116
Total number of timesteps:  124182
Shots with Complete NaN: 
85701 samples total
Removing ECH
Removed 46375 samples
39326 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 5176 samples
34150 samples remaining
Removing weird I-coils
Removed 7495 samples
26655 samples remaining
Removing NaN
Removed 0 samples
26655 samples remaining
Removing dudtrip
Removed 620 samples
26035 samples remaining
26035 samples remaining after pruning


Normalizing:  46%|####5     | 11/24 [00:00<00:00, 109.54it/s]

Total number of samples:  26035
Number of training samples:  23469
Number of validation samples:  2566


Normalizing: 100%|##########| 24/24 [00:00<00:00, 53.25it/s] 


In [12]:
"""THIS IS THE ONE FROM THE PAPER AS OF 3/5"""
file_path='/home/jabbate/run_results_12_11/model-conv2d_profiles-dens-temp-q_EFIT01-rotation-press_EFIT01_act-curr_target-pinj-tinj-target_density-bt_targ-dens-temp-q_EFIT01-rotation-press_EFIT01_profLB-0_actLB-6_ftop-True_11Dec19-18-30_Scenario-12.h5'

base_path = os.path.expanduser('/projects/EKOLEMEN/profile_predictor/run_results_03_10/')
"""good but overfit"""
#model_path = 'model-conv2d_profiles-dens-temp-q_EFIT01-rotation-press_EFIT01_act-target_density-pinj-tinj-curr_target_01Mar20-19-50_Scenario-93.h5'
"""good val but looks bad?"""
# model_path = 'model-conv2d_profiles-dens-temp-q_EFIT01-rotation-press_EFIT01_act-target_density-pinj-tinj-curr_target_01Mar20-02-31_Scenario-71.h5'

model_path = 'model-conv2d_profiles-dens-temp-q_EFIT01-rotation-press_EFIT01_act-target_density-pinj-tinj-curr_target_15Mar20-03-48_Scenario-265.h5'
file_path = base_path + model_path


model = keras.models.load_model(file_path, compile=False)
print('loaded model: ' + file_path.split('/')[-1])
file_path = file_path[:-3] + '_params.pkl'
with open(file_path, 'rb') as f:
    scenario = pickle.load(f, encoding='latin1')
print('loaded dict: ' + file_path.split('/')[-1])
print('with parameters: ' + str(scenario.keys()))

loaded model: model-conv2d_profiles-dens-temp-q_EFIT01-rotation-press_EFIT01_act-target_density-pinj-tinj-curr_target_15Mar20-03-48_Scenario-265.h5
loaded dict: model-conv2d_profiles-dens-temp-q_EFIT01-rotation-press_EFIT01_act-target_density-pinj-tinj-curr_target_15Mar20-03-48_Scenario-265_params.pkl
with parameters: dict_keys(['model_type', 'model_kwargs', 'input_profile_names', 'target_profile_names', 'sample_weightin', 'scalar_input_names', 'std_activation', 'batch_size', 'process_data', 'predict_deltas', 'epochs', 'loss_function', 'actuator_names', 'profile_downsample', 'sample_weighting', 'loss_function_kwargs', 'flattop_only', 'raw_data_path', 'invert_q', 'processed_filename_base', 'optimizer', 'optimizer_kwargs', 'shuffle_generators', 'pruning_functions', 'normalization_method', 'window_length', 'window_overlap', 'profile_lookback', 'actuator_lookback', 'lookahead', 'sample_step', 'uniform_normalization', 'train_frac', 'val_frac', 'val_idx', 'nshots', 'excluded_shots', 'lookbac

In [15]:
orig_data_path = '/scratch/gpfs/jabbate/full_data/train_data_full.pkl'
test_data_path = '/scratch/gpfs/jabbate/full_data_with_error/test_data.pkl' 
testdata, valdata, test_normalization_dict = helpers.data_generator.process_data(test_data_path,
                                                      scenario['sig_names'],
                                                      scenario['normalization_method'],
                                                      scenario['window_length'],
                                                      scenario['window_overlap'],
                                                      scenario['lookbacks'],
                                                      scenario['lookahead'],
                                                      scenario['sample_step'],
                                                      scenario['uniform_normalization'],
                                                      1, #scenario['train_frac'],
                                                      0, #scenario['val_frac'],
                                                      scenario['nshots'],
                                                      2, #scenario['verbose']
                                                      scenario['flattop_only'],
                                                      randomize=False,
                                                      pruning_functions=scenario['pruning_functions'],
                                                      excluded_shots = scenario['excluded_shots'],
                                                      delta_sigs = [],
                                                      invert_q=scenario.setdefault('invert_q',False),
                                                      val_idx = 1)

testdata = helpers.normalization.renormalize(helpers.normalization.denormalize(testdata.copy(),test_normalization_dict),scenario['normalization_dict'])

Loading
Signals: curr_target, dens, density_estimate, kappa_EFIT01, li_EFIT01, pinj, press_EFIT01, q_EFIT01, rotation, target_density, temp, tinj, triangularity_bot_EFIT01, triangularity_top_EFIT01, volume_EFIT01
Number of useable shots:  1116
Number of shots used:  1116
Total number of timesteps:  124182
Shots with Complete NaN: 
85701 samples total
Removing ECH
Removed 46375 samples
39326 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 5176 samples
34150 samples remaining
Removing weird I-coils
Removed 7495 samples
26655 samples remaining
Removing NaN
Removed 0 samples
26655 samples remaining
Removing dudtrip
Removed 620 samples
26035 samples remaining
26035 samples remaining after pruning


Normalizing:   0%|          | 0/24 [00:00<?, ?it/s]

Total number of samples:  26035
Number of training samples:  23469
Number of validation samples:  2566


Normalizing: 100%|##########| 24/24 [00:00<00:00, 54.81it/s]


In [16]:
orig_data_path = '/scratch/gpfs/jabbate/full_data/train_data_full.pkl'
test_data_path = '/scratch/gpfs/jabbate/full_data_with_error/test_data.pkl' 
testdata, valdata, test_normalization_dict = helpers.data_generator.process_data(test_data_path,
                                                      scenario['sig_names'],
                                                      scenario['normalization_method'],
                                                      scenario['window_length'],
                                                      scenario['window_overlap'],
                                                      scenario['lookbacks'],
                                                      scenario['lookahead'],
                                                      scenario['sample_step'],
                                                      scenario['uniform_normalization'],
                                                      1, #scenario['train_frac'],
                                                      0, #scenario['val_frac'],
                                                      scenario['nshots'],
                                                      2, #scenario['verbose']
                                                      scenario['flattop_only'],
                                                      randomize=False,
                                                      pruning_functions=scenario['pruning_functions'],
                                                      excluded_shots = scenario['excluded_shots'],
                                                      delta_sigs = [],
                                                      invert_q=scenario.setdefault('invert_q',False),
                                                      val_idx = 1)

testdata = helpers.normalization.renormalize(helpers.normalization.denormalize(testdata.copy(),test_normalization_dict),scenario['normalization_dict'])


Loading
Signals: curr_target, dens, density_estimate, kappa_EFIT01, li_EFIT01, pinj, press_EFIT01, q_EFIT01, rotation, target_density, temp, tinj, triangularity_bot_EFIT01, triangularity_top_EFIT01, volume_EFIT01
Number of useable shots:  1116
Number of shots used:  1116
Total number of timesteps:  124182
Shots with Complete NaN: 
85701 samples total
Removing ECH
Removed 46375 samples
39326 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 5176 samples
34150 samples remaining
Removing weird I-coils
Removed 7495 samples
26655 samples remaining
Removing NaN
Removed 0 samples
26655 samples remaining
Removing dudtrip
Removed 620 samples
26035 samples remaining
26035 samples remaining after pruning


Normalizing:   0%|          | 0/24 [00:00<?, ?it/s]

Total number of samples:  26035
Number of training samples:  23469
Number of validation samples:  2566


Normalizing: 100%|##########| 24/24 [00:00<00:00, 56.49it/s]


In [17]:
traindatafull = copy.deepcopy(traindata)
testdata = copy.deepcopy(traindata)
testdatafull = copy.deepcopy(testdata)

In [18]:
traindeltas = {sig:traindatafull[sig][:,-1,::2] - traindatafull[sig][:,0,::2] for sig in scenario['target_profile_names']}
traindata = {sig:traindatafull[sig][:,-1,::2] for sig in scenario['target_profile_names']}


In [19]:
testdeltas = {sig:testdatafull[sig][:,-1,::2] - testdatafull[sig][:,0,::2] for sig in scenario['target_profile_names']}
testdata = {sig:testdatafull[sig][:,-1,::2] for sig in scenario['target_profile_names']}
nbins=4
n = int(len(traindata['temp'])/nbins)

In [20]:
subdata = [{sig:traindata[sig][i*n:(i+1)*n] for sig in scenario['target_profile_names']} for i in range(nbins)]
subdata.insert(0,{sig:testdata[sig] for sig in scenario['target_profile_names']})

subdelta = [{sig:traindeltas[sig][i*n:(i+1)*n] for sig in scenario['target_profile_names']} for i in range(nbins)]
subdelta.insert(0,{sig:testdeltas[sig] for sig in scenario['target_profile_names']})

In [21]:
def scalarize_median(arr, **kwargs):
    return np.median(arr,axis=-1)

def scalarize_iqr(arr, **kwargs):
    return np.percentile(arr,75,axis=-1) - np.percentile(arr,25,axis=-1)

def scalarize_mean(arr, **kwargs):
    return np.mean(arr,axis=-1)

def scalarize_std(arr, **kwargs):
    return np.std(arr,axis=-1)

def scalarize_pca_1(arr, **kwargs):
    fitter = kwargs.get('fitter')
    ret = fitter.transform(arr).squeeze()[:,0]
    return ret

def scalarize_pca_2(arr, **kwargs):
    fitter = kwargs.get('fitter')
    ret = fitter.transform(arr).squeeze()[:,1]
    return ret

def scalarize_pca_3(arr, **kwargs):
    fitter = kwargs.get('fitter')
    ret = fitter.transform(arr).squeeze()[:,2]
    return ret

def scalarize_pca_4(arr, **kwargs):
    fitter = kwargs.get('fitter')
    ret = fitter.transform(arr).squeeze()[:,3]
    return ret

def scalarize_pca_5(arr, **kwargs):
    fitter = kwargs.get('fitter')
    ret = fitter.transform(arr).squeeze()[:,4]
    return ret

def scalarize_pca_6(arr, **kwargs):
    fitter = kwargs.get('fitter')
    ret = fitter.transform(arr).squeeze()[:,5]
    return ret

In [23]:
with open(os.path.expanduser('~/plasma-profile-predictor/pca_fitters.pkl'),'rb') as f:
    full_pca_fitter = pickle.load(f, encoding='latin1')
with open(os.path.expanduser('~/plasma-profile-predictor/pca_kernels.pkl'),'rb') as f:
    kernels = pickle.load(f, encoding='latin1')

FileNotFoundError: [Errno 2] No such file or directory: '/home/lzfang/plasma-profile-predictor/pca_fitters.pkl'

In [22]:
scalarize_functions=[scalarize_mean, scalarize_std, scalarize_pca_1,scalarize_pca_2, scalarize_median, scalarize_iqr]
scalarize_function_names=['mean','std','pca 1','pca 2','median','iqr']
profiles = ['dens','temp','rotation','q_EFIT01','press_EFIT01']


scalarizeddata = [{sig:{name: scalarize_fun(subdata[i][sig], fitter=full_pca_fitter[sig]) for name, scalarize_fun in zip(scalarize_function_names, scalarize_functions)}
               for sig in profiles} for i in range(len(subdata))]
scalarizeddelta = [{sig:{name: scalarize_fun(subdelta[i][sig], fitter=full_pca_fitter[sig]) for name, scalarize_fun in zip(scalarize_function_names, scalarize_functions)}
               for sig in profiles} for i in range(len(subdelta))]

NameError: name 'full_pca_fitter' is not defined

In [ ]:
inds = np.argwhere(scalarizeddelta[0]['dens']['iqr']>5)
normalization_dict['dens']
scalarize_iqr(subdelta[0]['dens'][1494])
testdatafull['shotnum'][inds][:,0,-1]
with open(orig_data_path,'rb') as f:
    orig_data = pickle.load(f,encoding='latin1')

In [ ]:
profiles = ['temp','dens','itemp','press_EFIT01','q_EFIT01']
psi = np.linspace(0,1,33)

In [ ]:
train_generator = DataGenerator(traindata,
                                1, #scenario['batch_size'],
                                scenario['input_profile_names'],
                                scenario['actuator_names'],
                                scenario['target_profile_names'],
                                scenario['scalar_input_names'],
                                scenario['lookbacks'],
                                scenario['lookahead'],
                                scenario['predict_deltas'],
                                scenario['profile_downsample'],
                                False,
                                sample_weights = 'std') #scenario['shuffle_generators'])

In [13]:
train_generator_batch = DataGenerator(traindatafull,
                                128, #scenario['batch_size'],
                                scenario['input_profile_names'],
                                scenario['actuator_names'],
                                scenario['target_profile_names'],
                                scenario['scalar_input_names'],
                                scenario['lookbacks'],
                                scenario['lookahead'],
                                scenario['predict_deltas'],
                                scenario['profile_downsample'],
                                shuffle=False,
                                sample_weights = None) #scenario['shuffle_generators'])

In [14]:
baseline = {sig:[] for sig in scenario['target_profile_names']}
predictions = {sig:[] for sig in scenario['target_profile_names']}
for i in range(len(train_generator_batch)):
    print(i,end=',')
    sample = train_generator_batch[i]
    out = model.predict(sample[0])
    for i, sig in enumerate(scenario['target_profile_names']):
        if scenario['predict_deltas']:
            baseline[sig].append(sample[1]['target_'+sig])
            predictions[sig].append(out[i])
            
        else:
            baseline[sig].append(sample[1]['target_'+sig].squeeze() - sample[0]['input_'+sig].squeeze())
            predictions[sig].append(out[i].squeeze() - sample[0]['input_'+sig].squeeze())
        
baseline = {sig:np.concatenate(baseline[sig],axis=0) for sig in scenario['target_profile_names']}
predictions = {sig:np.concatenate(predictions[sig],axis=0) for sig in scenario['target_profile_names']}

0,

KeyError: 'target_density'

In [ ]:
num_components=10
full_pca_fitters = {}
delta_pca_fitters = {}

for profile in profiles:
    full = np.array([train_generator[i][0]['input_' + profile] for i in range(len(train_generator))]).squeeze()
    delta = np.array([train_generator[i][1]['target_' + profile] for i in range(len(train_generator))]).squeeze()
    print(profile, ' made arrays')
    full_pca_fitters[profile] = decomposition.IncrementalPCA(n_components=num_components).fit(full)
    print(profile, ' done full')
    delta_pca_fitters[profile] = decomposition.IncrementalPCA(n_components=num_components).fit(delta)
    print(profile, ' done deltas')